In [8]:
import tensorflow as tf
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification
from transformers import InputExample, InputFeatures
import numpy as np
import pandas as pd
import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import collections
import pickle
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import os
from xgboost import XGBClassifier

os.environ['KMP_DUPLICATE_LIB_OK']='True'

/Users/spencermoon/Documents/sentiment/env/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Predicting from Loaded ALBERT Model

In [54]:
# model = TFAlbertForSequenceClassification.from_pretrained('albert-base-v1', num_labels=2)
# model.load_weights('../model/checkpoint/checkpoint')

# Load model and tokenizer
model= tf.keras.models.load_model('../model/trained-model', compile=True)
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v1')

# Check its architecture
model.summary()

Model: "tf_albert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
albert (TFAlbertMainLayer)   multiple                  11683584  
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 11,685,122
Trainable params: 11,685,122
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Create prediction value
prediction = ['Hello, this is great!']
prediction = pd.DataFrame({'tweet': prediction})

In [48]:
# Create function for turning individual data points into examples
def convert_to_examples(df): 
  tweet_input = df.apply(lambda x: InputExample(guid=None, text_a = x['tweet'], text_b = None, label = None), axis = 1)

  return tweet_input

# Create function for data ingestion pipeline
def convert_to_tf_dataset(examples, tokenizer, max_length=140):
  features = []
  counter = 0

  for e in examples:
    try:
      input_dict = tokenizer.encode_plus(
          e.text_a,
          add_special_tokens=True,
          max_length=max_length,
          return_token_type_ids=True,
          return_attention_mask=True,
          padding='max_length',
          truncation=True
      )
      input_ids, token_type_ids, attention_mask = (input_dict['input_ids'], input_dict['token_type_ids'], input_dict['attention_mask'])
      features.append(InputFeatures(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label))
    except:
      counter += 1
      pass
  
  print('data tokenization complete with %s data points unprocessed' % (counter))

    
  def gen():
    for f in features:
        yield ({'input_ids': f.input_ids, 'attention_mask': f.attention_mask, 'token_type_ids': f.token_type_ids}, f.label)

#   return tf.data.Dataset.from_generator(gen,
#                                         ({'input_ids': tf.int32, 'attention_mask': tf.int32, 'token_type_ids': tf.int32}, tf.int64),
#                                         ({'input_ids': tf.TensorShape([None]), 'attention_mask': tf.TensorShape([None]), 'token_type_ids': tf.TensorShape([None])} ,tf.TensorShape([]))
#                                         )

  return tf.data.Dataset.from_generator(gen,
                                        ({'input_ids': tf.int32}),
                                        ({'input_ids': tf.TensorShape([None, 5])})
                                        )

In [49]:
tweet_input = convert_to_examples(prediction)
tweet_data = convert_to_tf_dataset(list(tweet_input), tokenizer)

data tokenization complete with 0 data points unprocessed


In [50]:
model(tweet_data)

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (11 total):
    * <_VariantDataset shapes: {input_ids: (None, 5)}, types: {input_ids: tf.int32}>
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * False
  Keyword arguments: {}

Expected these arguments to match one of the following 2 option(s):

Option 1:
  Positional arguments (11 total):
    * {'input_ids': TensorSpec(shape=(None, 5), dtype=tf.int32, name='input_ids/input_ids')}
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * False
  Keyword arguments: {}

Option 2:
  Positional arguments (11 total):
    * {'input_ids': TensorSpec(shape=(None, 5), dtype=tf.int32, name='input_ids/input_ids')}
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * True
  Keyword arguments: {}

# Predicting from Loaded XGBoost Model

In [7]:
# Replace contractions
def replace_contraction(text):
    text = contractions.fix(text)
    
    return text

# Remove URLs
def remove_url(text):
    text = re.sub(r'http\S+', '', text)

    return text

# Tokenize tweets
def tokenize_tweet(text):
    tknzr = TweetTokenizer(preserve_case=False, strip_handles=True)
    text= tknzr.tokenize(text)
    
    return text

# Remove non-ASCII character
def remove_non_ascii(words):
    new_words = []
    for w in words:
        new_word = unicodedata.normalize('NFKD', w).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    
    return new_words

# Remove punctuation
def remove_punctuation(words):
    new_words = []
    for w in words:
        new_word = re.sub(r'[^\w\s]', '', w)
        if new_word != '':
            new_words.append(new_word)
    
    return new_words

# Replace numbers
def replace_number(words):
    p = inflect.engine()
    new_words = []
    for w in words:
        if w.isdigit():
            new_word = p.number_to_words(w)
            new_words.append(new_word)
        else:
            new_words.append(w)
    return new_words

# Remove stopwords
def remove_stopwords(words):
    new_words = []
    for w in words:
        if w not in stopwords.words('english'):
            new_words.append(w)
    return new_words

# Stem words
def stem_words(words):
    stemmer = LancasterStemmer()
    stems = []
    for w in words:
        stem = stemmer.stem(w)
        stems.append(stem)
    
    return stems

# Lemmatize verbs
def lemmatize_verbs(words):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for w in words:
        lemma = lemmatizer.lemmatize(w, pos='v')
        lemmas.append(lemma)
    
    return lemmas

# Combine normalizing functions
def normalize(words):
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = replace_number(words)
    words = remove_stopwords(words)
    words = stem_words(words)
    words = lemmatize_verbs(words)
    
    return words

# Create function for text preprocessing
def preprocess(text):
    text = remove_url(text)
    text = replace_contraction(text)
    words = tokenize_tweet(text)
    words = normalize(words)
    
    return words

In [9]:
# Process prediction value
prediction = 'Hello, this is great!'
text = preprocess(prediction)

['hello', 'gre']

In [10]:
# Load tree model
open_file = open('../model/xgboost.pkl', "rb")
model = pickle.load(open_file)
open_file.close()

In [11]:
# Load D2V model
d2v = Doc2Vec.load('../model/d2v.model')
text = d2v.infer_vector(text)
tweet = pd.DataFrame([text])

In [15]:
# Make prediction
model.predict_proba(tweet)

array([[0.11996794, 0.88003206]], dtype=float32)